In [4]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable

In [5]:
with open('../datasets/feature.txt') as f:
    lines = f.readlines()

In [6]:
colnames = []
for line in lines[1:]:
    line = line.split(",",2)[1]
    colnames.append(line)

In [38]:
print(colnames)

['srcip', 'sport', 'dstip', 'dsport', 'proto', 'state', 'dur', 'sbytes', 'dbytes', 'sttl', 'dttl', 'sloss', 'dloss', 'service', 'Sload', 'Dload', 'Spkts', 'Dpkts', 'swin', 'dwin', 'stcpb', 'dtcpb', 'smeansz', 'dmeansz', 'trans_depth', 'res_bdy_len', 'Sjit', 'Djit', 'Stime', 'Ltime', 'Sintpkt', 'Dintpkt', 'tcprtt', 'synack', 'ackdat', 'is_sm_ips_ports', 'ct_state_ttl', 'ct_flw_http_mthd', 'is_ftp_login', 'ct_ftp_cmd', 'ct_srv_src', 'ct_srv_dst', 'ct_dst_ltm', 'ct_src_ ltm', 'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm', 'attack_cat', 'Label']


In [20]:
unsw1 = pd.read_csv("../datasets/UNSW-NB15_1.csv", names=colnames, header=None)

/tmp/ipykernel_114373/3484622219.py:1: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  unsw1 = pd.read_csv("../datasets/UNSW-NB15_1.csv", names=colnames, header=None)


In [22]:
unsw1.head(1).values

array([['59.166.0.0', 1390, '149.171.126.6', '53', 'udp', 'CON',
        0.001055, 132, 164, 31, 29, 0, 0, 'dns', 500473.9375,
        621800.9375, 2, 2, 0, 0, 0, 0, 66, 82, 0, 0, 0.0, 0.0,
        1421927414, 1421927414, 0.017, 0.013, 0.0, 0.0, 0.0, 0, 0, 0, 0,
        0, 3, 7, 1, 3, 1, 1, 1, nan, 0]], dtype=object)

In [9]:
unsw2 = pd.read_csv("../datasets/UNSW-NB15_2.csv", names=colnames, header=None)
unsw2.shape

/tmp/ipykernel_114373/182996346.py:1: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  unsw2 = pd.read_csv("../datasets/UNSW-NB15_2.csv", names=colnames, header=None)


(700001, 49)

In [77]:
def preproccess_data(unsw):
    # Load data
    """Load unsw dataset"""
    y = unsw.iloc[:,-1:]
    #one-hot-encode parameters
    proto = pd.get_dummies(unsw['proto'])
    state = pd.get_dummies(unsw['state'])
    service = pd.get_dummies(unsw['service'])
    service = service.iloc[:,1:]
    raw_data = unsw.values

    #remove encoded parameters and add one hot
    for x in ['proto','srcip','dstip','state', 'service','attack_cat','sport', 'dsport']:
        unsw = unsw.drop(x,axis = 1)
    
    # unsw['sport'] = unsw['sport'].str.replace('0x000b','0')
    # unsw['sport'] = unsw['sport'].str.replace('0x000c','0')
    # unsw['sport'] = unsw['sport'].str.replace('0x20205321','0')
    # unsw['dsport'] = unsw['dsport'].str.replace('0xc0a8','0')
    unsw = unsw.replace('','0')
    
    for x in [proto, state, service]:
        unsw = unsw.join(x)
        
    X = unsw.iloc[:,:-2]
    return y, X

In [72]:
y1, X1 = preproccess_data(unsw1)

In [73]:
X1.shape

(700001, 202)

In [78]:
y2, X2 = preproccess_data(unsw2)

In [75]:
clf1 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                   random_state=13, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False).fit(X1,y1)

/home/tk/.local/lib/python3.10/site-packages/sklearn/utils/validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [79]:
clf2 = LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True, intercept_scaling=1,
                   l1_ratio=None, max_iter=100, multi_class='auto', n_jobs=None, penalty='l2', 
                   random_state=13, solver='lbfgs', tol=0.0001, verbose=0, warm_start=False).fit(X2,y2)

ValueError: could not convert string to float: ''

In [84]:
print(X2.isnull().sum())

dur       0
sbytes    0
dbytes    0
sttl      0
dttl      0
         ..
irc       0
pop3      0
radius    0
smtp      0
snmp      0
Length: 200, dtype: int64
